<a href="https://colab.research.google.com/github/FarukKaramanVektorel/yz/blob/master/txtToCsvForFineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import nltk
import re
import csv
import random
from nltk.tokenize import sent_tokenize, word_tokenize
import os

# NLTK veri dosyalarını indirme
print("NLTK veri dosyaları indiriliyor...")
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Google Colab'de çalışıp çalışmadığını kontrol etme
try:
    from google.colab import files
    is_colab = True
    print("Google Colab ortamı tespit edildi.")
except ImportError:
    is_colab = False
    print("Yerel ortam tespit edildi.")

# Dosya yükleme fonksiyonu
def upload_file():
    if is_colab:
        print("Lütfen metin dosyanızı yükleyin...")
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
    else:
        # Yerel ortam için dosya yolu isteme
        file_name = input("Lütfen metin dosyasının yolunu girin: ")
        if not os.path.exists(file_name):
            raise FileNotFoundError(f"Dosya bulunamadı: {file_name}")

    with open(file_name, "r", encoding="utf-8") as file:
        text = file.read()
    return text, file_name

# 1. Cümle tabanlı soru-cevap çiftleri oluşturma
def generate_sentence_qa_pairs(text):
    try:
        sentences = sent_tokenize(text)
    except LookupError:
        # Eğer sent_tokenize çalışmazsa, basit bir bölme kullan
        sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]

    qa_pairs = []

    for sentence in sentences:
        sentence = sentence.strip()
        if len(sentence) < 15:  # Çok kısa cümleleri atla
            continue

        # Cümleden farklı soru türleri oluşturma
        question_variations = [
            f"Bu cümlede ne anlatılıyor: '{sentence[:30]}...'?",
            f"'{sentence[:20]}...' cümlesi hakkında ne söyleyebilirsiniz?",
            f"Metinde geçen şu ifade ne anlama geliyor: '{sentence[:30]}...'?",
            f"Bu cümlede hangi önemli bilgi veriliyor?",
            f"'{sentence[:15]}...' ifadesi hangi bağlamda kullanılmıştır?"
        ]

        # Rastgele bir soru tipi seç
        question = random.choice(question_variations)
        qa_pairs.append((question, sentence))

    return qa_pairs

# 2. Anahtar kelime tabanlı soru-cevap çiftleri oluşturma
def extract_keywords(text):
    try:
        # Metni kelimelere ayırma
        words = word_tokenize(text.lower())

        # Durak kelimeleri (stopwords) çıkarma
        try:
            from nltk.corpus import stopwords
            try:
                stop_words = set(stopwords.words('turkish'))  # Türkçe için stopwords
            except:
                stop_words = set(stopwords.words('english'))  # Türkçe yoksa İngilizce kullan
        except:
            # Stopwords yüklenemezse, basit bir liste kullan
            stop_words = set(['ve', 'veya', 'ile', 'bu', 'bir', 'için', 'de', 'da', 'ki', 'the', 'and', 'or', 'a', 'an', 'in', 'on', 'at', 'to', 'for'])

        filtered_words = [word for word in words if word.isalnum() and word not in stop_words]

        # Kelimelerin sıklığını hesapla
        word_freq = {}
        for word in filtered_words:
            if len(word) > 2:  # Çok kısa kelimeleri atla
                word_freq[word] = word_freq.get(word, 0) + 1

        # En sık geçen kelimeleri al
        sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
        top_keywords = [word for word, freq in sorted_words[:20]]

        return top_keywords
    except Exception as e:
        print(f"Anahtar kelime çıkarma hatası: {e}")
        # Basit bir kelime listesi döndür
        words = re.findall(r'\b\w{3,}\b', text.lower())
        word_freq = {}
        for word in words:
            word_freq[word] = word_freq.get(word, 0) + 1
        sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
        return [word for word, freq in sorted_words[:20]]

def generate_keyword_qa_pairs(text, keywords):
    try:
        sentences = sent_tokenize(text)
    except:
        sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]

    qa_pairs = []

    for keyword in keywords:
        # Anahtar kelimeyi içeren cümleleri bulma
        relevant_sentences = [s for s in sentences if keyword.lower() in s.lower()]

        if relevant_sentences:
            # Bu anahtar kelime hakkında soru varyasyonları oluşturma
            question_variations = [
                f"Metinde '{keyword}' hakkında ne bilgi veriliyor?",
                f"'{keyword}' kelimesi metinde hangi bağlamda kullanılmış?",
                f"Bu metinde '{keyword}' kavramı nasıl açıklanıyor?",
                f"'{keyword}' ile ilgili metinde hangi örnekler verilmiş?",
                f"Metinde '{keyword}' ile ilişkili hangi konular ele alınmış?"
            ]

            # Rastgele bir soru seç
            question = random.choice(question_variations)
            answer = " ".join(relevant_sentences[:2])  # İlk iki ilgili cümleyi birleştir
            qa_pairs.append((question, answer))

    return qa_pairs

# 3. Paragraf tabanlı soru-cevap çiftleri oluşturma
def split_into_paragraphs(text):
    # Metni paragraflara ayırma
    paragraphs = re.split(r'\n\s*\n', text)
    return [p.strip() for p in paragraphs if len(p.strip()) > 50]

def generate_paragraph_qa_pairs(text):
    paragraphs = split_into_paragraphs(text)
    qa_pairs = []

    for i, paragraph in enumerate(paragraphs):
        # Paragrafın ilk birkaç kelimesini al
        words = paragraph.split()
        first_words = " ".join(words[:min(5, len(words))])

        # Paragraf için soru şablonları
        question_templates = [
            f"Metindeki {i+1}. paragraf ne anlatıyor?",
            f"'{first_words}...' ile başlayan paragrafta hangi bilgiler veriliyor?",
            f"Metinde '{first_words}...' ile başlayan kısım hakkında detaylı bilgi verir misiniz?",
            f"Dökümanın {i+1}. bölümünü özetler misiniz?",
            f"'{first_words}...' kısmında anlatılanları açıklar mısınız?"
        ]

        # Rastgele bir soru şablonu seç
        question = random.choice(question_templates)
        qa_pairs.append((question, paragraph))

    return qa_pairs

# 4. Genel sorular oluşturma
def generate_general_qa_pairs(text):
    paragraphs = split_into_paragraphs(text)
    qa_pairs = []

    # Genel sorular
    general_questions = [
        "Bu metnin ana konusu nedir?",
        "Bu metinde anlatılan en önemli bilgiler nelerdir?",
        "Bu metni kısaca özetleyebilir misiniz?",
        "Bu metinde hangi kavramlar ele alınıyor?",
        "Bu metnin amacı nedir?",
        "Bu metinde hangi görüşler savunuluyor?",
        "Bu metinde hangi örnekler verilmiş?",
        "Bu metinde hangi sonuçlara ulaşılmış?",
        "Bu metinde hangi öneriler sunulmuş?",
        "Bu metinde hangi karşılaştırmalar yapılmış?"
    ]

    # Genel sorular için cevap olarak ilk birkaç paragrafı kullan
    if paragraphs:
        for question in general_questions:
            answer = " ".join(paragraphs[:min(2, len(paragraphs))])
            qa_pairs.append((question, answer))
    else:
        # Paragraf yoksa, metni doğrudan kullan
        for question in general_questions[:3]:  # Sadece ilk 3 soruyu kullan
            answer = text[:min(500, len(text))]
            qa_pairs.append((question, answer))

    return qa_pairs

# 5. Tüm yöntemleri birleştirme
def generate_all_qa_pairs(text, max_pairs=100):
    all_qa_pairs = []

    # 1. Cümle tabanlı soru-cevap çiftleri
    print("Cümle tabanlı soru-cevap çiftleri oluşturuluyor...")
    sentence_qa_pairs = generate_sentence_qa_pairs(text)
    all_qa_pairs.extend(sentence_qa_pairs)
    print(f"{len(sentence_qa_pairs)} cümle tabanlı soru-cevap çifti oluşturuldu.")

    # 2. Anahtar kelime tabanlı soru-cevap çiftleri
    print("Anahtar kelime tabanlı soru-cevap çiftleri oluşturuluyor...")
    keywords = extract_keywords(text)
    keyword_qa_pairs = generate_keyword_qa_pairs(text, keywords)
    all_qa_pairs.extend(keyword_qa_pairs)
    print(f"{len(keyword_qa_pairs)} anahtar kelime tabanlı soru-cevap çifti oluşturuldu.")

    # 3. Paragraf tabanlı soru-cevap çiftleri
    print("Paragraf tabanlı soru-cevap çiftleri oluşturuluyor...")
    paragraph_qa_pairs = generate_paragraph_qa_pairs(text)
    all_qa_pairs.extend(paragraph_qa_pairs)
    print(f"{len(paragraph_qa_pairs)} paragraf tabanlı soru-cevap çifti oluşturuldu.")

    # 4. Genel sorular
    print("Genel soru-cevap çiftleri oluşturuluyor...")
    general_qa_pairs = generate_general_qa_pairs(text)
    all_qa_pairs.extend(general_qa_pairs)
    print(f"{len(general_qa_pairs)} genel soru-cevap çifti oluşturuldu.")

    # Çiftleri karıştırma ve en fazla max_pairs tanesini seçme
    random.shuffle(all_qa_pairs)
    return all_qa_pairs[:min(max_pairs, len(all_qa_pairs))]

# Ana fonksiyon
def main():
    print("Metin dosyasından soru-cevap çiftleri oluşturma aracı")
    print("----------------------------------------------------")

    try:
        # Dosya yükleme
        text, file_name = upload_file()
        print(f"Dosya başarıyla yüklendi: {file_name}")
        print(f"Metin uzunluğu: {len(text)} karakter")

        # Kaç soru-cevap çifti oluşturulacağını sor
        max_pairs = 100  # Varsayılan değer
        try:
            user_input = input(f"Kaç soru-cevap çifti oluşturmak istersiniz? (Varsayılan: {max_pairs}): ")
            if user_input.strip():
                max_pairs = int(user_input)
        except:
            print(f"Geçersiz giriş. Varsayılan değer kullanılacak: {max_pairs}")

        # Soru-cevap çiftleri oluşturma
        print(f"En fazla {max_pairs} soru-cevap çifti oluşturuluyor...")
        qa_pairs = generate_all_qa_pairs(text, max_pairs)

        # CSV dosyasına yazma
        output_file = "sc_ciftleri_tum_varyasyonlar.csv"
        with open(output_file, "w", encoding="utf-8", newline="") as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["Input)", "Output)"])
            for question, answer in qa_pairs:
                writer.writerow([question, answer])

        print(f"Toplam {len(qa_pairs)} soru-cevap çifti oluşturuldu ve {output_file} dosyasına kaydedildi.")

        # CSV dosyasını indirme
        if is_colab:
            files.download(output_file)
            print(f"CSV dosyası indirildi: {output_file}")
        else:
            print(f"CSV dosyası şu konuma kaydedildi: {os.path.abspath(output_file)}")

    except Exception as e:
        print(f"Bir hata oluştu: {e}")
        import traceback
        traceback.print_exc()

# Scripti çalıştır
if __name__ == "__main__":
    main()

NLTK veri dosyaları indiriliyor...
Google Colab ortamı tespit edildi.
Metin dosyasından soru-cevap çiftleri oluşturma aracı
----------------------------------------------------
Lütfen metin dosyanızı yükleyin...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Saving tapu_sicil_tuzugu.txt to tapu_sicil_tuzugu (2).txt
Dosya başarıyla yüklendi: tapu_sicil_tuzugu (2).txt
Metin uzunluğu: 56776 karakter
Kaç soru-cevap çifti oluşturmak istersiniz? (Varsayılan: 100): 1000
En fazla 1000 soru-cevap çifti oluşturuluyor...
Cümle tabanlı soru-cevap çiftleri oluşturuluyor...
335 cümle tabanlı soru-cevap çifti oluşturuldu.
Anahtar kelime tabanlı soru-cevap çiftleri oluşturuluyor...
Anahtar kelime çıkarma hatası: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CSV dosyası indirildi: qa_pairs_all_variations.csv
